In [ ]:
EURIAI_API_KEY = ""

In [5]:
from euriai.langchain import create_chat_model

chat_model = create_chat_model(
    api_key=EURIAI_API_KEY,
    model="gpt-4.1-nano",
    temperature=0.7
)

response = chat_model.invoke("What is artificial intelligence?")
print(response.content)

Artificial intelligence (AI) is the branch of computer science focused on creating systems and machines that can perform tasks typically requiring human intelligence. These tasks include learning from data, recognizing patterns, understanding language, solving problems, and making decisions. AI can be categorized into narrow AI, which is designed for specific tasks (like virtual assistants or recommendation systems), and general AI, which would possess the ability to understand, learn, and apply knowledge across a wide range of activities, similar to human intelligence.


In [6]:
chat_model = create_chat_model(
    api_key=EURIAI_API_KEY,
    model="gpt-4.1-nano",
    temperature=0.7
)

response = chat_model.invoke("What is Agentic AI?")
print(response.content)

Agentic AI refers to artificial intelligence systems designed to act autonomously and proactively to achieve specific goals, often exhibiting behaviors similar to agency or decision-making. Unlike passive AI that merely responds to inputs, agentic AI can initiate actions, adapt to new situations, and make decisions independently within predefined parameters. These systems are often used in applications such as autonomous agents, robots, virtual assistants, and complex problem-solving environments, where their ability to operate with a degree of independence enhances efficiency and effectiveness.


In [7]:
from langchain_core.tools import BaseTool, tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableWithMessageHistory
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    CombinedMemory,
    ReadOnlySharedMemory
)

In [ ]:
from euriai.langchain import create_chat_model

model = create_chat_model(
    api_key=EURIAI_API_KEY,
    model="gpt-4.1-nano",
    temperature=0.2
)

model

In [ ]:
def calculator(expression: str) -> str:
    """Evaluate a numeric math expression . supports +, -, *, /, **, parentheses 
    and all kind of mathematical function"""

In [12]:
import ast,re,math,os,sys
from euriai.langchain import create_chat_model
from langchain.agents import create_react_agent,AgentExecutor
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda,RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory ,InMemoryChatMessageHistory as ChatMessageHistory
from langchain_core.tools import BaseTool,tool
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    CombinedMemory,
    ReadOnlySharedMemory,
    ConversationEntityMemory

)

In [13]:
@tool("calculator",return_direct=True)
def calculator(expression: str) -> str:
    """Evaluate a numeric math expression . supprts + , -,*,/,**,parentheses and all kind of mathamatical functions"""
    allowed_nodes =(
        ast.Expression,
        ast.UnaryOp,
        ast.unaryop,
        ast.BinOp,
        ast.operator,
        ast.Num,
        ast.Load,
        ast.pow,
        ast.FunctionDef,
        ast.Module,
        ast.Expr,
        ast.Call,
        ast.Name,
        ast.arguments,
        ast.args,
        ast.Constant
    )
    
    allowed_names = {k:v for k,v in vars(math).items() if not k.startswith("_")}
    allowed_names.update({"abs": abs, "round": round,"min": min,"max": max})
    node = ast.parse(expression, mode="eval")
    
    
    for n in ast.walk(node):
        if not isinstance(n, allowed_nodes):
            raise ValueError(f"Expression contains invalid node {type(n)}")
        if isinstance(n, ast.Name) and n.id not in allowed_names:
            raise ValueError(f"Expression contains invalid name {n.id}")
    code = compile(node, "<string>", "eval")
    return str(eval(code, {"__builtins__": {}}, allowed_names))

In [14]:
tool_math = [calculator]
tool_math

[StructuredTool(name='calculator', description='Evaluate a numeric math expression . supprts + , -,*,/,**,parentheses and all kind of mathamatical functions', args_schema=<class 'langchain_core.utils.pydantic.calculator'>, return_direct=True, func=<function calculator at 0x00000154FE9ABC40>)]

In [15]:
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder

In [19]:
react_prompt_math = ChatPromptTemplate.from_messages([(
    "system","you are a precise math assistant , you can use thse tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final give me a numeric answer to the question"),
    ("human","questions: {input}\n{agent_scratchpad}")
])

In [20]:
create_react_agent(
    llm=model,
    prompt=react_prompt_math,
    tools=tool_math
)

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'tools': 'calculator(expression: str) -> str - Evaluate a numeric math expression . supprts + , -,*,/,**,parentheses and all kind of mathamatical functions', 'tool_names': 'calculator'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], input_types={}, partial_variables={}, template='you are a precise math assistant , you can use thse tools:\n{tools}\nwhen you are going to use this tools follw these instruction exactly the same format:\nQuestions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n alwasy finish with final give me a numeric answer to the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input']

In [22]:
memory_math = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
)

In [23]:
agent_math = create_react_agent(
    llm=model,
    tools=tool_math,
    prompt=react_prompt_math
)

In [24]:

math_executor = AgentExecutor(
    agent=agent_math,
    tools=tool_math,
    memory=memory_math,
    verbose=True,
    handle_parsing_errors=
        (
            "you did not follow the instruction that i have given you to use the tools"
        )
)

In [25]:
@tool("kb_search", return_direct=True)
def kb_search(query: str) -> str:
    """a mock function to search from a knowledge base"""
    knowledge_base = {
        "what is the capital of france": "The capital of France is Paris.",
        "who is the president of the united states": "The president of the United States is Joe Biden.",
        "what is the largest mammal": "The largest mammal is the blue whale.",
    }
    return knowledge_base.get(query.lower(), "I don't know the answer to that question.")

In [27]:
tools_kb = [kb_search]
tools_kb

[StructuredTool(name='kb_search', description='a mock function to search from a knowledge base', args_schema=<class 'langchain_core.utils.pydantic.kb_search'>, return_direct=True, func=<function kb_search at 0x00000154FEB04360>)]

In [28]:
react_prompt_kb = ChatPromptTemplate.from_messages([
    ("system","you are a helpful assistant that can answer question based on your knowledge base and you can use the following tools:\n{tools}\n"
    "when you are going to use this tools follw these instruction exactly the same format:\n"
    "Questions :......\nthought ...\nAction by using one of the tools access that you have [{tool_names}]\n "
    "alwasy finish with final give me a numeric answer to the question"),
    ("human","questions: {input}\n{agent_scratchpad}")
])

In [29]:
agent_kb = create_react_agent(
    llm=model,
    tools=tools_kb,
    prompt=react_prompt_kb
)

In [30]:
mem_kb = ConversationBufferWindowMemory(
    memory_key="chat_history",
    return_messages=True,
    input_key="input",
    output_key="output",
    k=5
)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_10048\4073518797.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  mem_kb = ConversationBufferWindowMemory(


In [31]:
kb_executor = AgentExecutor(
    agent=agent_kb,
    tools=tools_kb,
    memory=mem_kb,
    verbose=True,
    handle_parsing_errors=(
        "you did not follow the instruction that i have given you to use the kb  tools"
    ),
)

In [32]:
kb_executor = AgentExecutor(
    agent=agent_kb,
    tools=tools_kb,
    memory=mem_kb,
    verbose=True,
    handle_parsing_errors=(
        "you did not follow the instruction that i have given you to use the kb  tools"
    ),
)

In [ ]:
router_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a router. Read the user message and output exactly one token:
    - MATH if the user is asking for calculations
    - KB if the user is asking for general knowledge questions

    Output only KB or MATH, nothing else."""),
    ("human", "{input}")
])

In [72]:
def _dispatcher(inputs:dict):
    """it will take new inputs from human and it will also attach hitory from memory """
    user_msg = inputs["input"]
    choice = router_prompt.invoke({"input": user_msg}).strip().upper()
    if choice == "MATH":
        return math_executor.invoke({"input": user_msg})
    elif choice == "KB":
        return kb_executor.invoke({"input": user_msg})
    else:
        return "I can only answer math and kb related questions"

In [73]:
dispatcher = RunnableLambda(_dispatcher)
dispatcher

RunnableLambda(_dispatcher)

In [74]:
_sessions = {}

def _get_history(session_id:str):
    if session_id not in _sessions:
        _sessions[session_id] = ChatMessageHistory()
    return _sessions[session_id]

In [75]:
orchestrator = RunnableWithMessageHistory(
    runnable=dispatcher,
    get_session_history=_get_history,
    input_key="input",
    history_messages_key="history"
)

In [76]:
cfg = {"configurable": {"session_id": "user_1"}}